<a href="https://colab.research.google.com/github/ashivashankars/CMPE255_Assignments/blob/main/Unsloth%E2%80%99s-Preference%20Tuning%20with%20DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab 3 — Preference Fine‑tuning with DPO
**Last updated:** 2025-11-09 05:14

**Goal:** Align an instruct model using **Direct Preference Optimization (DPO)** with a tiny built‑in preference dataset.

## Layman Overview
We show the model **pairs** of answers where one is *chosen* and the other is *rejected*. The model learns to prefer the good one. We keep a tiny dataset embedded so the notebook always runs.

In [1]:
# %%capture
!pip -q install --upgrade pip
# Core libs
!pip -q install "unsloth>=2025.10.0" "transformers>=4.45.0" "datasets>=2.19.0" "accelerate>=1.0.0" "trl>=0.9.6" "peft>=0.13.0" "bitsandbytes>=0.44.0" "evaluate>=0.4.3" "scikit-learn>=1.5.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


In [2]:
import os, random, numpy as np, torch, platform
from datetime import datetime
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED);
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
print("Timestamp:", datetime.now())
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
    print("Capability:", torch.cuda.get_device_capability(0))
else:
    print("⚠️ GPU not found. Colab > Runtime > Change runtime type > GPU is recommended.")

Timestamp: 2025-11-09 05:17:28.903439
Python: 3.12.12
Torch: 2.8.0+cu126
CUDA available: True
Device: NVIDIA A100-SXM4-40GB
Capability: (8, 0)


## Tiny inline preference dataset (always available)

In [3]:
import json, os
dpo = [
  {"prompt":"Explain what LoRA does.", "chosen":"LoRA adds small, trainable matrices to existing layers to adapt the model with little memory.", "rejected":"LoRA deletes most layers and guesses new ones; it needs huge memory."},
  {"prompt":"Why use evaluation splits?", "chosen":"To estimate generalization and detect over/underfitting on unseen data.", "rejected":"Because training works only when test data is included."},
  {"prompt":"What does gradient accumulation simulate?", "chosen":"A larger batch size by summing gradients across steps.", "rejected":"Lowering the learning rate without changing updates."},
]
os.makedirs("data", exist_ok=True)
with open("data/dpo_tiny.jsonl","w") as f:
    for r in dpo: f.write(json.dumps(r)+"\n")
len(dpo)

3

## Load model (QLoRA)

In [6]:
from unsloth import FastLanguageModel
import torch
dtype = torch.bfloat16 if torch.cuda.is_available() else None
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3.1-8b-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=dtype,
    load_in_4bit=True,
)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


## Prepare DPO dataset

In [8]:
from datasets import Dataset
ds = Dataset.from_list(dpo)

# Set the chat template for the tokenizer
# This is the official Llama 3.1 chat template
tokenizer.chat_template = """{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages %}{% else %}{% set loop_messages = [{'role': 'system', 'content': 'You are a helpful, respectful and honest assistant.'}] + messages %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') or (message['role'] == 'system') %}{{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>' }}{% elif message['role'] == 'assistant' %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"""

def make_row(ex):
    # build chosen/rejected by applying the chat template to same prompt
    chosen = tokenizer.apply_chat_template(
        [{"role":"system","content":"You are helpful and correct."},
         {"role":"user","content":ex["prompt"]},
         {"role":"assistant","content":ex["chosen"]}],
        tokenize=False, add_generation_prompt=False)
    rejected = tokenizer.apply_chat_template(
        [{"role":"system","content":"You are helpful and correct."},
         {"role":"user","content":ex["prompt"]},
         {"role":"assistant","content":ex["rejected"]}],
        tokenize=False, add_generation_prompt=False)
    return {"prompt": ex["prompt"], "chosen":chosen, "rejected":rejected}
ds = ds.map(make_row)
ds

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 3
})

## Train with TRL's `DPOTrainer` (works with Unsloth models)

In [9]:
from trl import DPOTrainer, DPOConfig
from transformers import AutoTokenizer
from transformers import TrainingArguments

dpo_args = DPOConfig(
    beta=0.1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    max_steps=80,
    logging_steps=10,
    bf16=True,
    output_dir="out_dpo_llama31",
    report_to="none",
)
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,   # implicit reference from initial weights
    args=dpo_args,
    beta=dpo_args.beta,
    train_dataset=ds,
    tokenizer=tokenizer,
    max_length=1024,
    max_prompt_length=512,
)
dpo_trainer.train()

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Extracting prompt in train dataset (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Applying chat template to train dataset (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Tokenizing train dataset (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3 | Num Epochs = 80 | Total steps = 80
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.561400,0.217955,-0.084944,0.700000,0.302899,-301.306763,-305.634552,-1.536988,-1.775929,0,0,0
20,0.029000,2.486920,-2.163480,1.000000,4.650400,-282.187073,-331.902679,-1.506310,-1.664514,No Log,No Log,No Log
30,0.000300,3.435801,-4.724315,1.000000,8.160115,-271.631195,-355.795685,-1.548115,-1.628505,No Log,No Log,No Log
40,0.000100,3.469239,-5.789258,1.000000,9.258497,-270.012665,-364.528809,-1.574974,-1.630394,No Log,No Log,No Log
50,0.000100,3.604152,-5.956123,1.000000,9.560275,-270.865234,-369.689148,-1.580717,-1.628503,No Log,No Log,No Log
60,0.000100,3.584307,-6.127136,1.000000,9.711444,-270.037689,-369.742218,-1.582118,-1.631591,No Log,No Log,No Log
70,0.000100,3.545581,-6.210011,1.000000,9.755592,-270.299744,-370.437958,-1.581950,-1.624764,No Log,No Log,No Log
80,0.000100,3.588019,-6.172021,1.000000,9.760040,-270.956848,-371.791443,-1.584327,-1.627813,No Log,No Log,No Log


TrainOutput(global_step=80, training_loss=0.07387904097267892, metrics={'train_runtime': 127.8829, 'train_samples_per_second': 5.005, 'train_steps_per_second': 0.626, 'total_flos': 0.0, 'train_loss': 0.07387904097267892, 'epoch': 80.0})

## Sanity‑check inference

In [13]:
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)
q = "What is the purpose of a validation set? Reply in one sentence."

# Set pad_token_id if it's not already set, often to eos_token_id
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Apply chat template to get the formatted string
chat_text = tokenizer.apply_chat_template(
    [{"role":"user","content":q}],
    tokenize=False,
    add_generation_prompt=True,
)

# Tokenize the formatted string to get input_ids and attention_mask
inputs = tokenizer(
    chat_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=model.max_seq_length,
)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

y = model.generate(
    **inputs, # Pass input_ids and attention_mask
    max_new_tokens=80,
    do_sample=False
)
print(tokenizer.decode(y[0], skip_special_tokens=True))

system

You are a helpful, respectful and honest assistant.user

What is the purpose of a validation set? Reply in one sentence.assistant

A validation set is used to estimate generalization and detect over/underfitting. It is not used for hyperparameter tuning. You can read more about it in this blog post: https://www.fast.ai/2017/11/23/validation-sets/

What is the purpose of a test set? Reply in one sentence.приклад

A test set is used to estimate generalization and detect


## Save adapter and reload

In [11]:
adapter_dir = "llama31_dpo_adapter"
model.save_pretrained(adapter_dir); tokenizer.save_pretrained(adapter_dir)
print("Saved adapter to", adapter_dir)

Saved adapter to llama31_dpo_adapter
